# ANALYSE SUR LE MODELE TARGET + XGBOOST

In [ ]:
import pandas as pd
import numpy as np
import panel as pn

from pandas_profiling import ProfileReport

from joblib import dump, load

import matplotlib.pyplot as plt
import seaborn as sns

import eli5
from eli5.sklearn import PermutationImportance

from pdpbox import pdp, get_dataset, info_plots

import shap

import lime
import lime.lime_tabular

In [ ]:
path = './data/'

## ANALYSE DES DONNEES TEST (PANDAS PROFILING)

In [19]:
test = pd.read_csv(path+'test_set3.csv')
test.head()

,nb_requete,index,order_requests,city,date,language,mobile,avatar_id,hotel_id,stock,group,brand,parking,pool,children_policy
0,1,0,1,vilnius,21,romanian,0,1,161,46,Boss Western,J.Halliday Inn,1,0,0
1,1,1,1,vilnius,21,romanian,0,1,187,32,Accar Hotels,Marcure,1,1,0
2,1,2,1,vilnius,21,romanian,0,1,279,12,Independant,Independant,1,0,0
3,1,3,1,vilnius,21,romanian,0,1,395,10,Accar Hotels,Ibas,0,0,0
4,1,4,1,vilnius,21,romanian,0,1,488,42,Accar Hotels,Safitel,1,1,1


In [20]:
#On affecte le bon type aux variables qualitatives
test["city"]=pd.Categorical(test["city"],ordered=False)
test["language"]=pd.Categorical(test["language"],ordered=False)
test["group"]=pd.Categorical(test["group"],ordered=False)
test["brand"]=pd.Categorical(test["brand"],ordered=False)

test["mobile"]=pd.Categorical(test["mobile"],ordered=False)
test["parking"]=pd.Categorical(test["parking"],ordered=False)
test["pool"]=pd.Categorical(test["pool"],ordered=False)
test["children_policy"]=pd.Categorical(test["children_policy"],ordered=False)

test["hotel_id"]=pd.Categorical(test["hotel_id"],ordered=False)
test["avatar_id"]=pd.Categorical(test["avatar_id"],ordered=False)

In [21]:
testP = test.drop(columns=["index","order_requests","avatar_id"])

In [ ]:
#ProfileReport(testP,minimal=True)
ProfileReport(testP)

## ANALYSE DES DONNEES REQUEST (PANDAS PROFILING)

In [2]:
data = pd.read_csv(path+'new_result_requetes.csv')
data.head()

In [24]:
#On affecte le bon type aux variables qualitatives
data["city"]=pd.Categorical(data["city"],ordered=False)
data["language"]=pd.Categorical(data["language"],ordered=False)
data["group"]=pd.Categorical(data["group"],ordered=False)
data["brand"]=pd.Categorical(data["brand"],ordered=False)

data["mobile"]=pd.Categorical(data["mobile"],ordered=False)
data["parking"]=pd.Categorical(data["parking"],ordered=False)
data["pool"]=pd.Categorical(data["pool"],ordered=False)
data["children_policy"]=pd.Categorical(data["children_policy"],ordered=False)

data["hotel_id"]=pd.Categorical(data["hotel_id"],ordered=False)
data["avatar_id"]=pd.Categorical(data["avatar_id"],ordered=False)

data["price"]=pd.DataFrame(data["price"], dtype=float)

In [25]:
dataP = data.drop(columns=["order_requests","avatar_id"])

In [ ]:
#ProfileReport(dataP,minimal=True)
ProfileReport(dataP)

## INTERPRETABILITE

### Preprocessing

In [ ]:
Y = data[["price"]]
    
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

In [ ]:
feature_names = X_test.columns
feature_names

In [ ]:
#model = joblib.load('./XGB_Target_model_saved.joblib')
#model = joblib.load('./XGB11_Target_model_saved_Final.joblib')

In [ ]:
model = booster.load_model(pip, './XGB11_Target_model_saved_Final.json')

In [ ]:
model_name = "XGBoost"

### Permutation Importance

In [ ]:
features_importance_dict = {}
permutation_impor = PermutationImportance(model, random_state=42).fit(X_test, y_test)

In [ ]:
plt.figure()
features_importance = {'Feature_name':feature_names, 'Importance':permumtation_impor.feature_importances_}  
features_importance = pd.DataFrame(features_importance) 
features_importance = features_importance.sort_values(['Importance'], ascending=False) 
features_importance_dict[name] = features_importance
ax = sns.barplot(x="Importance", y="Feature_name", data=features_importance)
plt.title(model_name)

In [ ]:
#sans l'hotel_id
ax = sns.barplot(x="Importance", y="Feature_name", data=data=features_importance[1:])
plt.title(model_name)

### PDP and ICE plots

In [ ]:
#pour combien de features?
top_features = features_importance_dict[model_name].Feature_name[:7].values
for i, feature in enumerate(top_features, 1):
  pdp_feat = pdp.pdp_isolate(model=model, dataset=X_test, model_features=feature_names, feature=feature)
  pdp.pdp_plot(pdp_feat, feature, plot_lines=True, frac_to_plot=0.1, figsize=(10,5))

In [ ]:
features_to_plot = ['stock','price']
inter1 = pdp.pdp_interact(model=model, dataset=X_test, model_features=feature_names, features=features_to_plot)

pdp.pdp_interact_plot(pdp_interact_out=inter1, feature_names=features_to_plot, plot_type='contour')
plt.show()

In [ ]:
features_to_plot = ['date','price']
inter1 = pdp.pdp_interact(model=model, dataset=X_test, model_features=feature_names, features=features_to_plot)

pdp.pdp_interact_plot(pdp_interact_out=inter1, feature_names=features_to_plot, plot_type='contour')
plt.show()

### SHAP

In [ ]:
shap.initjs() #needed to plot results directly on the notebook

idx = 1 # index of the instance we want to explain

explainer = shap.KernelExplainer(model.predict, X_test)

In [ ]:
#shap_values = explainer.shap_values(X_test.iloc[0,:])
#shap.force_plot(explainer.expected_value, shap_values, X_test.iloc[0,:])

#To speed up we just compute the shap values for 100? exemples
shap_values = explainer.shap_values(X_test.iloc[0:100,:])
shap.summary_plot(shap_values, X_test.iloc[0:100,:])

### LIME

In [ ]:
index = 0
explainer = lime.lime_tabular.LimeTabularExplainer(X_test.values, feature_names=feature_names, mode="regression")
exp = explainer.explain_instance(X_test.iloc[index], model.predict, num_features=7, top_labels=1)
exp.show_in_notebook(show_table=True, show_all=True)